## Notebook # 1

## Extracción de datos estática

*Acceso al sitio*

In [ ]:
%pip install beautifulsoup4

In [ ]:
#Importar librerías.
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
import datetime
import pickle

In [ ]:
#URL de Fuente: TLC Trip Record Data.
tlc_url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

#Solicitar URL y obtener objeto de respuesta.
tlc_respuesta = requests.get(tlc_url)

#Analizar el texto obtenido.
tlc_sopa = BeautifulSoup(tlc_respuesta.text, 'html.parser')

#Encontrar todos los hipervínculos presentes en la página web.
tlc_vinculos = tlc_sopa.find_all('a')

## Descarga

In [ ]:
#Descarga inicial.
#Yellow Taxi Trip Records.

años = ['2022']

i = 0
d_amarillo = {}
dh_amarillo = {}

for vinculo in tlc_vinculos:
    if(
        ('yellow' in vinculo.get('href', [])) and
        ('.parquet' in vinculo.get('href', []))
    ):
        for año in años:
            if (año in vinculo.get('href', [])):
                i += 1
                hipervinculo = vinculo.get('href', [])
                mes = hipervinculo[69]+hipervinculo[70]
                print('Elemento', i, 'leído.')
                #Leer el vínculo y asignarlo a un diccionario.
                d_amarillo["yellow_{0}_{1}".format(año, mes)] = "yellow_{0}_{1} = pd.read_parquet('{2}')".format(año, mes, hipervinculo)
                print('Elemento', i, 'escrito en diccionario.')

print('Todos los elementos se escribieron en el diccionario amarillo.')

*Acceso a Blob Storage* 

In [ ]:
#Datos de acceso
storage_account_name = "pfcrudo"
storage_account_key = "ASY9nV2+pddQN2Y8AUZef5m4JtdidWOa5RW+fNIkj2HE60F5s95ueJraFC2/Jv1KulbyL5hwcYk6+AStCn7cjw=="

In [ ]:
#Variable de configuración Spark
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)

Descarga de todo lo contenido en el diccionario

In [ ]:
#Permitir a Apache Arrow convertir Pandas a pySpark DF:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
container = "yellowtaxis"
blobstorage = f"wasbs://{container}@{storage_account_name}.blob.core.windows.net"

for variable in d_amarillo.values():
    exec(variable)

    for clave, valor in d_amarillo.items():
        if variable == valor:

            #Código para exportar al almacenamiento previo (antes de transformar)
            dataframe = spark.createDataFrame(vars()[clave])
            
            dataframe.write.format("parquet").save(f"wasbs://yellowtaxis@pfcrudo.blob.core.windows.net/{clave}.parquet")

            print(clave, "exportado a .parquet")

In [ ]:
#Exportar archivos pickles de los diccionarios de referencia al directorio del almacenamiento conveniente.

directorio_diccionarios = "/dbfs/d_amarillo.pickle"

amarillo = open(directorio_diccionarios, 'wb')
pickle.dump(d_amarillo, amarillo)
amarillo.close()

## Notebook 2

## Extracción de datos automática

### Acceso al sitio

In [ ]:
%pip install beautifulsoup4

In [ ]:
#Importar librerías.
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
import datetime
import pickle

In [ ]:
#URL de Fuente: TLC Trip Record Data
tlc_url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

#Solicitar URL y obtener objeto de respuesta.
tlc_respuesta = requests.get(tlc_url)

#Analizar el texto obtenido.
tlc_sopa = BeautifulSoup(tlc_respuesta.text, 'html.parser')

#Encontrar todos los hipervínculos presentes en la página web.
tlc_vinculos = tlc_sopa.find_all('a')

In [ ]:
#Leer diccionarios de referencia.
directorio_diccionarios = "/dbfs/d_amarillo.pickle"
d_amarillo = pd.read_pickle(directorio_diccionarios)

### Descarga de nuevos archivos

Acceso a Blob Storage

In [ ]:
#Datos de acceso
storage_account_name = "pfcrudo"
storage_account_key = "ASY9nV2+pddQN2Y8AUZef5m4JtdidWOa5RW+fNIkj2HE60F5s95ueJraFC2/Jv1KulbyL5hwcYk6+AStCn7cjw=="
container = "yellowtaxis"

In [ ]:
#Variable de configuración Spark
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)

Descarga


In [ ]:
#Permitir a Apache Arrow convertir Pandas a pySpark DF:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
#Obtener el año en curso.
date = datetime.date.today()
año = '{0}'.format(date.year)

In [ ]:
#Comparar con diccionario actual. Agregar (y exportar) sólo nueva variable.
for vinculo in tlc_vinculos:
    if(
        'yellow' in vinculo.get('href', []) and
        '.parquet' in vinculo.get('href', [])
    ):
        if (
            año in vinculo.get('href', [])
        ):
            hipervinculo = vinculo.get('href', [])
            mes = hipervinculo[69]+hipervinculo[70]

            #Leer el vínculo y compararlo con las otras entradas del diccionario.

            for clave, valor in d_amarillo.items():
                valor_año = valor[7]+valor[8]+valor[9]+valor[10]
                valor_mes = valor[13]+valor[14]
                
                if (
                    (año == valor_año) and
                    (mes == valor_mes)
                ):
                    equal = True
                    break
                else:
                    equal = False

            if (equal == False):
                #Agregar entrada nueva al diccionario.
                d_amarillo["yellow_{0}_{1}".format(año, mes)] = "yellow_{0}_{1} = pd.read_parquet('{2}')".format(año, mes, hipervinculo)

                #Exportar diccionario actualizado a almacenamiento conveniente.

                amarillo = open(directorio_diccionarios, 'wb')
                pickle.dump(d_amarillo, amarillo)
                amarillo.close()

                #Exportar nuevo archivo parquet (tabla) a almacenamiento previo (raw).

                #Ejecutar el comando
                exec(d_amarillo["yellow_{0}_{1}".format(año, mes)])

                #Pasar dataframe a parquet en la ubicación

                dataframe = spark.createDataFrame(vars()[list(d_amarillo)[-1]])

                dataframe.write.format("parquet").save(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/yellow_{año}_{mes}.parquet")

## Notebook #3

## Tubería con Auto Loader

*Acceso a Blob Storage*:

In [ ]:
storage_account_name = "pfcrudo"
storage_account_key = "ASY9nV2+pddQN2Y8AUZef5m4JtdidWOa5RW+fNIkj2HE60F5s95ueJraFC2/Jv1KulbyL5hwcYk6+AStCn7cjw=="

In [ ]:
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)

In [ ]:
#listado de archivos dentro del container
%fs ls wasbs://yellowtaxis@pfcrudo.blob.core.windows.net/

## Leer archivo y obtener el esquema general

In [ ]:
container = "yellowtaxis"
dataframe = spark.read.parquet(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/yellow_2021_12.parquet")

In [ ]:
#Probar si se ve el esquema
dataframe.schema

In [ ]:
#Probar si se ve el tipo de objeto
type(dataframe.schema)

## Establecer el esquema deseado (para reducir memoria):

In [ ]:
from pyspark.sql.types import StructType, StructField, ByteType, TimestampType, ShortType, StringType, FloatType

Exportar el esquema del archivo a JSON:

In [ ]:
#Guardar el schema a un archivo JSON en el almacenamiento de Databricks.
import json
with open("/dbfs/taxischema.json", "w") as f: 
    json.dump(dataframe.schema.jsonValue(), f)

In [ ]:
#Probar si se lee el esquema desde el archivo JSON.
with open("/dbfs/taxischema.json") as sch_file:
    dataset_schema = StructType.fromJson(json.load(sch_file))
    #print(dataset_schema.simpleString())
dataset_schema

### Lectura

Autoloader

In [ ]:
stream_in = (spark.readStream
                .format("cloudFiles")
                .option("cloudFiles.useNotifications", False)
                .option("cloudFiles.format", "parquet")
                .schema(dataset_schema)
                .load("wasbs://yellowtaxis@pfcrudo.blob.core.windows.net/"))

### Transformación

In [ ]:
from pyspark.sql.functions import col, year, mode, abs

In [ ]:
#Eliminar columnas indeseadas y rellenar nulos.
df = (
    stream_in
    .drop("VendorID", "RatecodeID", "store_and_fwd_flag")
    .na.fill(value = 0, subset = ["fare_amount", "extra", "mta_tax", "tolls_amount", "improvement_surcharge", "congestion_surcharge"])
)

In [ ]:
#Ignorar filas con ID de ubicación desconocida ("PULocationID", "DOLocationID", 264, 265).
df = df.filter(
    (col("PULocationID") != 264) & 
    (col("PULocationID") != 265) & 
    (col("DOLocationID") != 264) & 
    (col("DOLocationID") != 265)

In [ ]:
#Eliminar filas con distancia == 0 o > 250
df=df.filter(col("trip_distance")!=0)

df=df.filter(col("trip_distance")<250)

In [ ]:
#Usar valores absolutos para columnas numéricas.
df = df.withColumn("passenger_count", abs(col("passenger_count")))
df = df.withColumn("trip_distance", abs(col("trip_distance")))
df = df.withColumn("fare_amount", abs(col("fare_amount")))
df = df.withColumn("total_amount", abs(col("total_amount")))
df = df.withColumn("extra", abs(col("extra")))
df = df.withColumn("mta_tax", abs(col("mta_tax")))
df = df.withColumn("tip_amount", abs(col("tip_amount")))
df = df.withColumn("tolls_amount", abs(col("tolls_amount")))
df = df.withColumn("improvement_surcharge", abs(col("improvement_surcharge")))
df = df.withColumn("congestion_surcharge", abs(col("congestion_surcharge")))
df = df.withColumn("airport_fee", abs(col("airport_fee")))

### Carga a SQL Database

Conexión a SQL Database:

def output_sqldb(batch_df, batch_id):
    #Unas últimas transformaciones.
    right_year = batch_df.select(mode(year('tpep_pickup_datetime'))).head()[0]
    batch_df = batch_df.filter(year("tpep_pickup_datetime") == right_year)

    #Propiedades para la conexión.
    jdbcHostname = "pfservidor.database.windows.net"
    jdbcPort = 1433
    jdbcDatabase = "yellowtaxis"
    jdbcUsername = "ivnng"
    jdbcPassword = "henryPF2023"
    jdbcDriver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

    jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};databaseName={jdbcDatabase};user={jdbcUsername};password={jdbcPassword}"

    dbtable = "trips"

    #Convertir de PySpark DataFrame a Azure DW...
    batch_df.write.format("jdbc").mode("append").option("url", jdbcUrl).option("dbtable", dbtable).save()


Auto Loader

In [ ]:
stream_out = (df.writeStream
                .trigger(once = True)
                .foreachBatch(output_sqldb)
                .option("checkpointLocation", "wasbs://checkpoint@pfcrudo.blob.core.windows.net/")
                .start())